# 0. Install Dependencies

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame

# 1. Test Random Evirontment with OpenAI Gym

In [ ]:
import random
import gym

In [ ]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
states

In [ ]:
actions

In [ ]:
episodes = 10 
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.randint(0,1)
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode {} : reward : {}".format(episode, score))

# 2. Create a DeepLearning Model with Keras

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(states, actions):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(1, states)))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

# 4. Save Model's Weights

In [ ]:
dqn.save_weights('dqn_CartPole_weights.h5f', overwrite=True)